In [1]:
from typing import *

import torch 
import numpy as np 
import shutil 
import json 
import zipfile 
import io 
import pytorch_lightning as pl 
from scipy.sparse import csc_matrix 
from pathlib import Path 
import torch.nn.functional as F
from torchmetrics.functional import accuracy, precision, recall 
from pytorch_tabnet.tab_network import TabNet
import copy
import warnings

import sys, os 
sys.path.append('../src')

from torchmetrics.functional import *
import torchmetrics 

from data import *
from lightning_train import *
from model import *

from torchmetrics.functional.classification.stat_scores import _stat_scores_update, _stat_scores
from sklearn.metrics import classification_report
from bigcsv.bigcsv import experimental_to_h5ad 

In [9]:
labels = pd.read_csv('../data/benchmark/mouse_labels.csv')

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_42970/4063021729.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  labels = pd.read_csv('../data/benchmark/mouse_labels.csv')


In [12]:
labels['region_label'].unique()

array(['VISp', 'ALM', 'SSp', 'MOp', 'ACA', 'HIP', 'PL-ILA', 'ORB', 'SSs',
       'RSP', 'AI', 'TEa-PERI-ECT', 'ENTm', 'RSPv', 'ENTl',
       'PAR-POST-PRE', 'SUB-ProS', 'PTLp', 'AUD', 'GU', 'VIS', 'CLA'],
      dtype=object)

In [14]:
visp_labels = labels[labels['region_label'] == 'VISp']
visp_labels.index.name = 'cell'

In [16]:
visp_labels.to_csv('../data/benchmark/visp_labels.csv')

In [17]:
pd.read_csv('../data/benchmark/visp_labels.csv')

,cell,sample_name,donor_sex_id,donor_sex_label,donor_sex_color,region_id,region_label,region_color,platform_label,cluster_order,...,cell_type_alt_alias_id,cell_type_alias_id,cell_type_accession_id,cell_type_designation_color,cell_type_alt_alias_color,cell_type_alias_color,cell_type_accession_color,cortical_layer_label,cortical_layer_order,cortical_layer_color
0,0,US-1250273_E1_S37,1,F,#565353,1,VISp,#9299FF,SS,259,...,259,259,259,#286291,#286291,#286291,#286291,L4/5/6,11,#7373FF
1,1,US-1250273_E2_S01,2,M,#ADC4C3,1,VISp,#9299FF,SS,259,...,259,259,259,#286291,#286291,#286291,#286291,L4/5/6,11,#7373FF
2,2,US-1250273_E2_S02,2,M,#ADC4C3,1,VISp,#9299FF,SS,259,...,259,259,259,#286291,#286291,#286291,#286291,L4/5/6,11,#7373FF
3,3,US-1250273_E2_S03,2,M,#ADC4C3,1,VISp,#9299FF,SS,197,...,197,197,197,#4EA8AC,#4EA8AC,#4EA8AC,#4EA8AC,L4/5/6,11,#7373FF
4,4,US-1250273_E2_S04,2,M,#ADC4C3,1,VISp,#9299FF,SS,245,...,245,245,245,#0D5D7E,#0D5D7E,#0D5D7E,#0D5D7E,L4/5/6,11,#7373FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15567,68481,SM-GE5US_S067_E1-50,2,M,#ADC4C3,1,VISp,#9299FF,SS,168,...,168,168,168,#02F970,#02F970,#02F970,#02F970,L2/3,6,#00CC44
15568,68482,SM-GE5US_S068_E1-50,2,M,#ADC4C3,1,VISp,#9299FF,SS,168,...,168,168,168,#02F970,#02F970,#02F970,#02F970,L2/3,6,#00CC44
15569,68483,SM-GE5US_S069_E1-50,2,M,#ADC4C3,1,VISp,#9299FF,SS,168,...,168,168,168,#02F970,#02F970,#02F970,#02F970,L2/3,6,#00CC44
15570,72577,SM-GE5PH_S383_E1-50,2,M,#ADC4C3,1,VISp,#9299FF,SS,112,...,112,112,112,#B9342C,#B9342C,#B9342C,#B9342C,L5/6,13,#6E1799


In [19]:
data = pd.read_csv('../data/benchmark/mouse_data.csv', nrows=4)
data

,sample_name,0610005C13Rik,0610006L08Rik,0610007P14Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610009O20Rik,0610010B08Rik,0610010F05Rik,...,n-R5s134,n-R5s136,n-R5s138,n-R5s139,n-R5s141,n-R5s142,n-R5s143,n-R5s144,n-R5s146,n-R5s149
0,US-1250273_E1_S37,0,0,32,31,0,10,0,0,50,...,0,0,0,0,0,0,0,0,0,0
1,US-1250273_E2_S01,0,1,83,54,0,2,13,0,98,...,0,0,0,0,0,0,0,0,0,0
2,US-1250273_E2_S02,0,0,37,31,0,8,21,0,109,...,0,0,0,0,0,0,0,0,0,0
3,US-1250273_E2_S03,0,1,162,49,32,0,33,0,96,...,0,0,0,0,0,0,0,0,0,0


In [2]:
experimental_to_h5ad(
    file='../data/benchmark/human.csv',
    outfile='../data/benchmark/human.h5ad',
    sep=',',
    sparsify=True,
    chunksize=100,
    index_col='sample_name'
)

Working on chunk 0/494
Working on chunk 1/494
Working on chunk 2/494
Working on chunk 3/494
Working on chunk 4/494
Working on chunk 5/494
Working on chunk 6/494
Working on chunk 7/494
Working on chunk 8/494
Working on chunk 9/494
Working on chunk 10/494
Working on chunk 11/494
Working on chunk 12/494
Working on chunk 13/494
Working on chunk 14/494
Working on chunk 15/494
Working on chunk 16/494
Working on chunk 17/494
Working on chunk 18/494
Working on chunk 19/494
Working on chunk 20/494
Working on chunk 21/494
Working on chunk 22/494
Working on chunk 23/494
Working on chunk 24/494
Working on chunk 25/494
Working on chunk 26/494
Working on chunk 27/494
Working on chunk 28/494
Working on chunk 29/494
Working on chunk 30/494
Working on chunk 31/494
Working on chunk 32/494
Working on chunk 33/494
Working on chunk 34/494
Working on chunk 35/494
Working on chunk 36/494
Working on chunk 37/494
Working on chunk 38/494
Working on chunk 39/494
Working on chunk 40/494
Working on chunk 41/494
Wo

Working on chunk 333/494
Working on chunk 334/494
Working on chunk 335/494
Working on chunk 336/494
Working on chunk 337/494
Working on chunk 338/494
Working on chunk 339/494
Working on chunk 340/494
Working on chunk 341/494
Working on chunk 342/494
Working on chunk 343/494
Working on chunk 344/494
Working on chunk 345/494
Working on chunk 346/494
Working on chunk 347/494
Working on chunk 348/494
Working on chunk 349/494
Working on chunk 350/494
Working on chunk 351/494
Working on chunk 352/494
Working on chunk 353/494
Working on chunk 354/494
Working on chunk 355/494
Working on chunk 356/494
Working on chunk 357/494
Working on chunk 358/494
Working on chunk 359/494
Working on chunk 360/494
Working on chunk 361/494
Working on chunk 362/494
Working on chunk 363/494
Working on chunk 364/494
Working on chunk 365/494
Working on chunk 366/494
Working on chunk 367/494
Working on chunk 368/494
Working on chunk 369/494
Working on chunk 370/494
Working on chunk 371/494
Working on chunk 372/494


In [2]:
data = an.read_h5ad('../data/benchmark/mouse_data.h5ad')

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [10]:
pd.read_csv('../data/benchmark/human.csv', nrows=5)


,sample_name,3.8-1.2,3.8-1.3,3.8-1.4,3.8-1.5,5-HT3C2,A1BG,A1BG-AS1,A1CF,A2M,...,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3,bA255A11.4,bA395L14.12
0,F2S4_160113_027_A01,0,0,0,0,0,0,0,0,41,...,0,282,0,0,0,41,448,0,0,1
1,F2S4_160113_027_B01,0,0,0,0,0,118,0,0,0,...,0,892,0,0,125,0,0,748,0,0
2,F2S4_160113_027_C01,0,0,0,0,0,0,0,0,0,...,0,0,1,0,161,0,102,0,0,0
3,F2S4_160113_027_D01,0,0,0,0,0,159,0,0,0,...,0,155,1,0,81,0,3,1,0,0
4,F2S4_160113_027_E01,0,0,0,0,0,1,0,0,1,...,0,14,1,0,490,0,4,33,0,0


In [7]:
labels = pd.read_csv('../data/benchmark/human_labels.csv')
labels.shape

(49417, 41)

In [6]:
data = pd.read_csv('../data/benchmark/mouse_data.csv')


KeyboardInterrupt



In [35]:
raw = pd.read_csv('../data/benchmark/mouse_data.csv', nrows=5, index_col='sample_name')

In [36]:
raw

,0610005C13Rik,0610006L08Rik,0610007P14Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610009O20Rik,0610010B08Rik,0610010F05Rik,0610010K14Rik,...,n-R5s134,n-R5s136,n-R5s138,n-R5s139,n-R5s141,n-R5s142,n-R5s143,n-R5s144,n-R5s146,n-R5s149
sample_name,,,,,,,,,,,,,,,,,,,,,
US-1250273_E1_S37,0,0,32,31,0,10,0,0,50,40,...,0,0,0,0,0,0,0,0,0,0
US-1250273_E2_S01,0,1,83,54,0,2,13,0,98,53,...,0,0,0,0,0,0,0,0,0,0
US-1250273_E2_S02,0,0,37,31,0,8,21,0,109,52,...,0,0,0,0,0,0,0,0,0,0
US-1250273_E2_S03,0,1,162,49,32,0,33,0,96,24,...,0,0,0,0,0,0,0,0,0,0
US-1250273_E2_S04,0,1,59,27,0,0,38,0,117,52,...,0,0,0,0,0,0,0,0,0,0


In [39]:
np.linalg.norm(raw.values - data.X[0:5].todense())

0.0